In [ ]:
import itertools
import numpy as np
from PIL import Image

def split_image_with_overlap(image_path, num_splits=6, overlap_percent=0.2):
    
    image = Image.open(image_path)
    width, height = image.size

    split_width = width // 3
    split_height = height // 2

    overlap_x = int(split_width * overlap_percent)
    overlap_y = int(split_height * overlap_percent)

    splits = []
    for i, j in itertools.product(range(3), range(2)):
        x1 = max(0, i * split_width - overlap_x)
        y1 = max(0, j * split_height - overlap_y)
        x2 = min(width, (i + 1) * split_width + overlap_x)
        y2 = min(height, (j + 1) * split_height + overlap_y)

        split = image.crop((x1, y1, x2, y2))
        splits.append((split, (x1, y1, x2, y2)))

    return splits

def adjust_bbox(bbox, original_coords, new_image_size):
   
    x1, y1, x2, y2 = bbox
    ox1, oy1, ox2, oy2 = original_coords
    new_width, new_height = new_image_size
    
    new_x1 = x1 + ox1
    new_y1 = y1 + oy1
    new_x2 = x2 + ox1
    new_y2 = y2 + oy1
    
    new_x1 = max(0, min(new_x1, new_width))
    new_y1 = max(0, min(new_y1, new_height))
    new_x2 = max(0, min(new_x2, new_width))
    new_y2 = max(0, min(new_y2, new_height))
    
    return (new_x1, new_y1, new_x2, new_y2)

def merge_predictions(original_image_path, split_predictions):

    original_image = Image.open(original_image_path)
    original_size = original_image.size
    
    merged_predictions = []
    for split_coords, bboxes in split_predictions:
        for bbox in bboxes:
            adjusted_bbox = adjust_bbox(bbox, split_coords, original_size)
            merged_predictions.append(adjusted_bbox)
    
    return merged_predictions

def non_max_suppression(boxes, overlap_thresh=0.5):

    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = boxes[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= overlap_thresh)[0]
        order = order[inds + 1]

    return boxes[keep].tolist()

if __name__ == "__main__":
    image_path = "path/to/your/image.jpg"
    splits = split_image_with_overlap(image_path)
    

    split_predictions = [
        (splits[0][1], [(10, 10, 50, 50, 0.9), (30, 30, 70, 70, 0.8)]),
        (splits[1][1], [(20, 20, 60, 60, 0.85)]),
    ]
    
    merged_predictions = merge_predictions(image_path, split_predictions)
    final_predictions = non_max_suppression(merged_predictions)
    
